In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!tar xf spark-3.5.2-bin-hadoop3.tar
!pip install -q findspark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.2-bin-hadoop3"

In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [5]:
# Read the dataset into a Spark DataFrame named df
df = spark.read.csv("geolocation.csv", header=True, inferSchema=True)

In [6]:
df.show(5)

+-------+--------+---------+---------+-----------+----------+----------+--------+---------+----------+
|truckid|driverid|    event| latitude|  longitude|      city|     state|velocity|event_ind|idling_ind|
+-------+--------+---------+---------+-----------+----------+----------+--------+---------+----------+
|    A54|     A54|   normal|38.440467|-122.714431|Santa Rosa|California|      17|        0|         0|
|    A20|     A20|   normal|36.977173|-121.899402|     Aptos|California|      27|        0|         0|
|    A40|     A40|overspeed|37.957702| -121.29078|  Stockton|California|      77|        1|         0|
|    A31|     A31|   normal|39.409608|-123.355566|   Willits|California|      22|        0|         0|
|    A71|     A71|   normal|33.683947|-117.794694|    Irvine|California|      43|        0|         0|
+-------+--------+---------+---------+-----------+----------+----------+--------+---------+----------+
only showing top 5 rows



In [7]:
df.printSchema()

root
 |-- truckid: string (nullable = true)
 |-- driverid: string (nullable = true)
 |-- event: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- velocity: integer (nullable = true)
 |-- event_ind: integer (nullable = true)
 |-- idling_ind: integer (nullable = true)



In [8]:
df.select("truckid", "driverid", "event", "city").filter("event != 'normal'").show()

+-------+--------+--------------------+-------------+
|truckid|driverid|               event|         city|
+-------+--------+--------------------+-------------+
|    A40|     A40|           overspeed|     Stockton|
|     A5|      A5|unsafe following ...|      Klamath|
|    A48|     A48|           overspeed|    Roseville|
|    A94|     A94|unsafe following ...|  San Quentin|
|    A49|     A49|      lane departure|        Ukiah|
|    A97|     A97|           overspeed|   Santa Rosa|
|    A75|     A75|      lane departure|    San Dimas|
|     A9|      A9|unsafe tail distance|    San Diego|
|    A80|     A80|unsafe tail distance|San Francisco|
|    A91|     A91|unsafe following ...|   Santa Rosa|
|    A59|     A59|unsafe following ...| Santa Monica|
|    A85|     A85|      lane departure|San Francisco|
|    A94|     A94|           overspeed| San Fernando|
|    A73|     A73|           overspeed|  Bakersfield|
|    A71|     A71|      lane departure|       Irvine|
|    A28|     A28|      lane

In [9]:
df.groupBy("event").count().show()

+--------------------+-----+
|               event|count|
+--------------------+-----+
|      lane departure|  152|
|unsafe tail distance|   65|
|              normal| 7543|
|unsafe following ...|  150|
|           overspeed|   90|
+--------------------+-----+



In [10]:
df.describe("velocity").show()

+-------+------------------+
|summary|          velocity|
+-------+------------------+
|  count|              8000|
|   mean|           35.0565|
| stddev|24.863750391090154|
|    min|                 0|
|    max|               100|
+-------+------------------+



In [11]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [12]:
!wget https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
!tar -xzf hadoop-3.3.6.tar.gz
!tar xf apache-hive-3.1.3-bin.tar

--2025-02-26 05:59:59--  https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.208.237, 135.181.214.104, 2a01:4f9:3a:2c57::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.208.237|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 730107476 (696M) [application/x-gzip]
Saving to: ‘hadoop-3.3.6.tar.gz’

hadoop-3.3.6.tar.gz 100%[===================>] 696.28M  27.7MB/s    in 26s     

2025-02-26 06:00:25 (27.0 MB/s) - ‘hadoop-3.3.6.tar.gz’ saved [730107476/730107476]



In [13]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/content/hadoop-3.3.6"
os.environ["HIVE_HOME"] = "/content/apache-hive-3.1.3-bin"

In [14]:
!rm -rf metastore_db

In [15]:
!$HIVE_HOME/bin/schematool -initSchema -dbType derby


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Metastore connection URL:	 jdbc:derby:;databaseName=metastore_db;create=true
Metastore Connection Driver :	 org.apache.derby.jdbc.EmbeddedDriver
Metastore connection User:	 APP
Starting metastore schema initialization to 3.1.0
Initialization script hive-schema-3.1.0.derby.sql

 
 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 

 

 
 
 

 
 
 


 
 
 

 
 
 


 
 
 
 
 
 
 


 
 
 

 
 
 

 
 

 
 
 
 
 

 
 
 
 
 
 


In [16]:
!$HIVE_HOME/bin/hive -e 'show databases;'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 3777b8b4-577f-4608-b767-ca5465864ab4

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 0df05612-ccf3-42fa-9e08-3f1976140bac
OK
default
Time taken: 1.715 seconds, Fetched: 1 row(s)


In [17]:
!$HIVE_HOME/bin/hive -e 'use default;'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = f3ba6346-4e97-4b4d-bd22-4eb8818f55a4

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 88cfab36-5f1e-4e02-97b3-01a2a0f11cb5
OK
Time taken: 1.429 seconds


In [18]:
%%bash
$HIVE_HOME/bin/hive -e "
CREATE TABLE default.amaht_trucks
(driverid String,truckid String,model String,
jun13_miles int,jun13_gas int,may13_miles int,may13_gas int,apr13_miles int,
apr13_gas int,mar13_miles int,mar13_gas int,feb13_miles int,feb13_gas int,
jan13_miles int,jan13_gas int,dec12_miles int,dec12_gas int,nov12_miles int,
nov12_gas int,oct12_miles int,oct12_gas int,sep12_miles int,sep12_gas int,
aug12_miles int,aug12_gas int,jul12_miles int,jul12_gas int,jun12_miles int,
jun12_gas int,may12_miles int,may12_gas int,apr12_miles int,apr12_gas int,
mar12_miles int,mar12_gas int,feb12_miles int,feb12_gas int,jan12_miles int,
jan12_gas int,dec11_miles int,dec11_gas int,nov11_miles int,nov11_gas int,
oct11_miles int,oct11_gas int,sep11_miles int,sep11_gas int,aug11_miles int,
aug11_gas int,jul11_miles int,jul11_gas int,jun11_miles int,jun11_gas int,
may11_miles int,may11_gas int,apr11_miles int,apr11_gas int,mar11_miles int,
mar11_gas int,feb11_miles int,feb11_gas int,jan11_miles int,jan11_gas int,
dec10_miles int,dec10_gas int,nov10_miles int,nov10_gas int,oct10_miles int,
oct10_gas int,sep10_miles int,sep10_gas int,aug10_miles int,aug10_gas int,
jul10_miles int,jul10_gas int,jun10_miles int,jun10_gas int,may10_miles int,
may10_gas int,apr10_miles int,apr10_gas int,mar10_miles int,mar10_gas int,
feb10_miles int,feb10_gas int,jan10_miles int,jan10_gas int,dec09_miles int,
dec09_gas int,nov09_miles int,nov09_gas int,oct09_miles int,oct09_gas int,
sep09_miles int,sep09_gas int,aug09_miles int,aug09_gas int,jul09_miles int,
jul09_gas int,jun09_miles int,jun09_gas int,may09_miles int,may09_gas int,
apr09_miles int,apr09_gas int,mar09_miles int,mar09_gas int,feb09_miles int,
feb09_gas int,jan09_miles int,jan09_gas int)
COMMENT 'trucks'
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LINES TERMINATED BY '\n'
STORED AS TEXTFILE
TBLPROPERTIES('skip.header.line.count'='1');
"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = fbccae81-f43a-4782-9067-183b8454a2c2

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 01528007-0c1e-4384-9b03-9130e5f05fc0
OK
Time taken: 2.713 seconds


In [19]:
%%bash
$HIVE_HOME/bin/hive -e "
CREATE TABLE default.amaht_geolocation (
truckid String,driverid String,event String,latitude String,longitude String,
city String,state String,velocity int,event_ind int,idling_ind int)
COMMENT 'geolocation'
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LINES TERMINATED BY '\n'
STORED AS TEXTFILE
TBLPROPERTIES('skip.header.line.count'='1');
"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 02ac6b63-14dd-4349-b370-73392e5afd5d

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 01567818-6560-44a6-a90a-895794e75391
OK
Time taken: 2.302 seconds


In [20]:
!$HIVE_HOME/bin/hive -e "show tables;"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = f84f419c-b341-42f2-838a-de0c7171c586

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = cc584754-5455-4d6a-9f75-b43b29adf449
OK
amaht_geolocation
amaht_trucks
Time taken: 1.64 seconds, Fetched: 2 row(s)


In [21]:
!$HIVE_HOME/bin/hive -e "DESCRIBE default.amaht_trucks;"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = bfbce6a2-4378-49eb-bfa8-fca7f10646ea

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 4c5c7eab-36fc-4858-be74-983182781d27
OK
driverid            	string              	                    
truckid             	string              	                    
model               	string              	                    
jun13_miles         	int                 	           

In [22]:
!$HIVE_HOME/bin/hive -e "DESCRIBE default.amaht_geolocation;"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = d92b5c01-c4f5-4473-9e05-33807cf333d8

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = e4fde536-c7f4-42ea-a9c9-5e6fe6fc3ffe
OK
truckid             	string              	                    
driverid            	string              	                    
event               	string              	                    
latitude            	string              	           

In [23]:
!$HADOOP_HOME/bin/hadoop fs -ls /user/hive/warehouse

Found 2 items
drwxr-xr-x   - root root       4096 2025-02-26 06:02 /user/hive/warehouse/amaht_geolocation
drwxr-xr-x   - root root       4096 2025-02-26 06:01 /user/hive/warehouse/amaht_trucks


In [24]:
!$HIVE_HOME/bin/hive -e "LOAD DATA LOCAL INPATH 'trucks_extended.csv' INTO TABLE default.amaht_trucks;"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 08ce5d0b-d37a-4662-8d0e-a8675bd955e5

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = a7cad244-92a8-47e4-b09a-1fcfe3e1ee55
Loading data to table default.amaht_trucks
OK
Time taken: 3.053 seconds


In [25]:
!$HIVE_HOME/bin/hive -e "LOAD DATA LOCAL INPATH 'geolocation.csv' INTO TABLE default.amaht_geolocation;"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 5c99b9c9-94be-4460-9454-db0ba0ba60e2

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 2aefd2ea-c8e2-4ed1-8096-544a38622701
Loading data to table default.amaht_geolocation
OK
Time taken: 3.197 seconds


In [26]:
!$HIVE_HOME/bin/hive -e "SELECT * FROM default.amaht_trucks LIMIT 1;"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = e3b961b2-1f92-4c40-b187-2a1bd2e7659a

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 1d807551-8174-432b-b707-1c79b8c85e32
OK
A1	A1	Freightliner	9217	1914	8769	1892	14234	3008	11519	2262	8676	1596	10025	1878	12647	2331	10214	2054	10807	2134	11127	2191	9754	1967	12925	2578	15792	3313	9052	1878	11062	2150	13594	2824	11019	2324	9222	1902	8565	1745	10410	1943	13450	3047

In [27]:
!$HIVE_HOME/bin/hive -e "SELECT * FROM default.amaht_geolocation LIMIT 5;"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 03fb1239-45f6-4d15-9a58-e367d384bb2d

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = a806d654-41f4-4a0f-ae8d-72f0ff7a1719
OK
A54	A54	normal	38.440467	-122.714431	Santa Rosa	California	17	0	0
A20	A20	normal	36.977173	-121.899402	Aptos	California	27	0	0
A40	A40	overspeed	37.957702	-121.29078	Stockton	California	77	1	0
A31	A31	normal	39.409608	-123.355566	Willits	Cali

In [28]:
%%bash
$HIVE_HOME/bin/hive -e "
CREATE TABLE default.amaht_truck_mileage STORED AS ORC AS
SELECT truckid, driverid, rdate, miles, gas, miles / gas mpg
FROM default.amaht_trucks
LATERAL VIEW stack(54,
'jun13',jun13_miles,jun13_gas,'may13',may13_miles,may13_gas,
'apr13',apr13_miles,apr13_gas,'mar13',mar13_miles,mar13_gas,
'feb13',feb13_miles,feb13_gas,'jan13',jan13_miles,jan13_gas,
'dec12',dec12_miles,dec12_gas,'nov12',nov12_miles,nov12_gas,
'oct12',oct12_miles,oct12_gas,'sep12',sep12_miles,sep12_gas,
'aug12',aug12_miles,aug12_gas,'jul12',jul12_miles,jul12_gas,
'jun12',jun12_miles,jun12_gas,'may12',may12_miles,may12_gas,
'apr12',apr12_miles,apr12_gas,'mar12',mar12_miles,mar12_gas,
'feb12',feb12_miles,feb12_gas,'jan12',jan12_miles,jan12_gas,
'dec11',dec11_miles,dec11_gas,'nov11',nov11_miles,nov11_gas,
'oct11',oct11_miles,oct11_gas,'sep11',sep11_miles,sep11_gas,
'aug11',aug11_miles,aug11_gas,'jul11',jul11_miles,jul11_gas,
'jun11',jun11_miles,jun11_gas,'may11',may11_miles,may11_gas,
'apr11',apr11_miles,apr11_gas,'mar11',mar11_miles,mar11_gas,
'feb11',feb11_miles,feb11_gas,'jan11',jan11_miles,jan11_gas,
'dec10',dec10_miles,dec10_gas,'nov10',nov10_miles,nov10_gas,
'oct10',oct10_miles,oct10_gas,'sep10',sep10_miles,sep10_gas,
'aug10',aug10_miles,aug10_gas,'jul10',jul10_miles,jul10_gas,
'jun10',jun10_miles,jun10_gas,'may10',may10_miles,may10_gas,
'apr10',apr10_miles,apr10_gas,'mar10',mar10_miles,mar10_gas,
'feb10',feb10_miles,feb10_gas,'jan10',jan10_miles,jan10_gas,
'dec09',dec09_miles,dec09_gas,'nov09',nov09_miles,nov09_gas,
'oct09',oct09_miles,oct09_gas,'sep09',sep09_miles,sep09_gas,
'aug09',aug09_miles,aug09_gas,'jul09',jul09_miles,jul09_gas,
'jun09',jun09_miles,jun09_gas,'may09',may09_miles,may09_gas,
'apr09',apr09_miles,apr09_gas,'mar09',mar09_miles,mar09_gas,
'feb09',feb09_miles,feb09_gas,'jan09',jan09_miles,jan09_gas )
dummyalias AS rdate, miles, gas;
"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = d7aa8c68-165e-437c-8fab-a848adad4e52

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 61503811-3125-4be6-9900-d03e4a256ae8
Query ID = root_20250226060457_273d973b-43f0-4dcd-ab38-e2c24a48f639
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks is set to 0 since there's no reduce operator
Job running in-process (local Hadoop)
2025-02-26 06:05:03,639 Stage-1

In [29]:
!$HIVE_HOME/bin/hive -e "show tables;"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = d7f517b4-9758-45b5-beb0-397e7d3560bc

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 29b5ceff-b95e-4c4e-a282-9b6318c568f2
OK
amaht_geolocation
amaht_truck_mileage
amaht_trucks
Time taken: 1.768 seconds, Fetched: 3 row(s)


In [30]:
!$HIVE_HOME/bin/hive -e "DESCRIBE default.amaht_truck_mileage;"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 341ed8d5-252c-472f-baa7-d54fcce80fd4

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = b5c23714-87b1-4002-8fb4-36f1dde3aaf9
OK
truckid             	string              	                    
driverid            	string              	                    
rdate               	string              	                    
miles               	int                 	           

In [31]:
!$HIVE_HOME/bin/hive -e "SET hive.cli.print.header=true;"
!$HIVE_HOME/bin/hive -e "SELECT * FROM default.amaht_truck_mileage LIMIT 10;"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 950fc259-6391-477c-aac6-551de53cd5c0

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = aaf2f37d-0481-4f7a-855d-87358b8f200f
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.

In [32]:
%%bash
$HIVE_HOME/bin/hive -e "
CREATE TABLE default.amaht_avg_mileage
STORED AS ORC
AS
SELECT truckid, avg(mpg) avgmpg
FROM default.amaht_truck_mileage
GROUP BY truckid
DISTRIBUTE BY avgmpg
SORT BY avgmpg DESC;
"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = b04cb875-10bd-49cf-b0bc-247f3a23a37c

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 445b1969-7045-4abf-b71f-8ed4c9b81661
Query ID = root_20250226060638_6510eb5e-d358-4127-b441-1617c4a0465c
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  s

In [33]:
!$HIVE_HOME/bin/hive -e "SELECT * FROM default.amaht_avg_mileage LIMIT 10;"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = db8cdda0-759a-4900-a57d-91ffab5eed6b

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = b23f6bc2-133b-4c83-8470-6b0435ddc178
OK
A18	6.295206724472384
A61	6.238571478738324
A79	6.230205355202399
A30	6.202007499297939
A60	5.96129692348809
A38	5.917100754501856
A13	5.8205454797114795
A40	5.8199316380329105
A2	5.795686564164179
A92	5.787692965941835
Time taken: 5.48 secon

In [34]:
%%bash
$HIVE_HOME/bin/hive -e "
CREATE TABLE default.amaht_driver_mileage
STORED AS ORC AS
SELECT driverid, SUM(miles) AS total_miles
FROM default.amaht_truck_mileage
GROUP BY driverid
SORT BY total_miles DESC;
"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 028aca14-dd06-4988-b3b6-ae3fe3745294

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 3f17f690-5fe6-4ceb-989e-a1d3dc12ae06
Query ID = root_20250226060727_927efa38-b61a-4528-ba93-6e952705483e
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  s

In [35]:
%%bash
$HIVE_HOME/bin/hive -e "
DESCRIBE FORMATTED default.amaht_driver_mileage;
"

# col_name            	data_type           	comment             
driverid            	string              	                    
total_miles         	bigint              	                    
	 	 
# Detailed Table Information	 	 
Database:           	default             	 
OwnerType:          	USER                	 
Owner:              	root                	 
CreateTime:         	Wed Feb 26 06:07:36 UTC 2025	 
LastAccessTime:     	UNKNOWN             	 
Retention:          	0                   	 
Location:           	file:/user/hive/warehouse/amaht_driver_mileage	 
Table Type:         	MANAGED_TABLE       	 
Table Parameters:	 	 
	COLUMN_STATS_ACCURATE	{\"BASIC_STATS\":\"true\"}
	bucketing_version   	2                   
	numFiles            	1                   
	numRows             	100                 
	rawDataSize         	9400                
	totalSize           	725                 
	transient_lastDdlTime	1740550057          
	 	 
# Storage Information	 	 
SerDe Library:      	or

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 8904d980-c2a9-43d9-ab03-a6e76ce4ceba

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 019af16e-cdf5-47b5-b5ab-8cec1bce3d79
OK
Time taken: 2.47 seconds, Fetched: 32 row(s)


In [36]:
%%bash
$HIVE_HOME/bin/hive -e "
SELECT driverid, total_miles
FROM default.amaht_driver_mileage
ORDER BY total_miles DESC
LIMIT 10;
"

A52	684131
A80	681115
A94	676750
A5	676574
A10	675377
A24	675334
A68	672089
A71	672026
A27	670601
A31	668620


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 45f7dbb1-4eed-4517-8215-79efc19260cd

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = d83e7666-58b1-48c5-a3b8-0be658db33d3
Query ID = root_20250226060815_f48ea0e1-da1a-4175-bdc7-bd18c49bf922
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks determined at compile time: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reduce